<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js048_windTunnelRLG2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js048_windTunnelRLG2D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.11.02 created,  last updated on 2024.01.25
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js048_windTunnelRLG2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js048_windTunnelRLG2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.06.26 created, last updated on 2018.11.20
//    ver 0.0.1  2019.01.18 v1, last updated on 2021.06.08
//    ver 0.0.2  2021.11.01 v2, last updated on 2021.11.01
//    ver 0.0.3  2023.03.29 v3, last updated on 2023.06.24
//
// --------------------  real-coded lattice gas model 2D
//
// (1) move particles - particle move in free without collision
//     r = r + v*dt
//
// (2) collision - all particles collide each other in the same cell
//     vi: velocity of i-th RLG particle after collision,
//     vm: mean velocity in the cell,
//     vi0; initial velocity of i-th RLG particle
//
//     vi = vm + rotate(theta)(vi0-vm),  (theta = PI/2 or -PI/2 : minmum viscosity)
//
// --------------------
*/

const windTunnelRLG2D = (function(){ // ====================  windTunnelRLG2D Module  ====================

	let g_NNmx = 96;								// x-max mean cell group 5x5 cells
	let g_NNmy = 48;								// y-max mean cell group 5x5 cells
	let g_Nsx = g_NNmx*5;							// x-max number of cell[Nsx][Nsy]
	let g_Nsy = g_NNmy*5;							// y-max number of cell[Nsx][Nsy]
	let g_NNs = 30;									// n-max number of section[Nsx][Nsy][NNs]
	let g_NinCell = 4;								// number of particles in the cell
	let g_NNp = g_Nsx*g_Nsy*g_NinCell;				// total number of particlesy
	let g_sysTime = 0.0;							// system time : dimensionless time
	let g_timeStep = 1.0;							// time step dt
	let g_dx = 1.0;									// x-space step  : dimensionless length
	let g_dy = 1.0;									// y-space step
	let g_xMax = g_Nsx*g_dx;						// x- box size
	let g_yMax = g_Nsy*g_dy;						// y- box size
	let g_gravity = 0.0;							// gravitational force direction is -y direction
	let g_vx0 = 0.3;								// fan - mean velocity

	const g_xx = dim1( g_NNp );						// x-component of particle position
	const g_yy = dim1( g_NNp );						// y-component of particle position
	const g_vx = dim1( g_NNp );						// x-component of particle velocity
	const g_vy = dim1( g_NNp );						// y-component of particle velocity
	const g_section = dimInt3( g_Nsx, g_Nsy, g_NNs );	//section[Nsx][Nsy][NNs]  particle number in cell[i][j]
	const g_cellAttribute = dimInt2( g_Nsx, g_Nsy );	//attribute at cell[i][j]
	const g_refTemp = [0,0,0,0];					// reference temp. refTemp[ca] : ca = cell attribute
	const g_meanDens = dim2( g_NNmx, g_NNmy );		// mean density
	const g_meanVx = dim2( g_NNmx, g_NNmy );		// mean flow vx
	const g_meanVy = dim2( g_NNmx, g_NNmy );		// mean flow vy
	const g_meanTemp = dim2( g_NNmx, g_NNmy );		// mean temperature
	const g_meanPres = dim2( g_NNmx, g_NNmy );		// mean pressure
	const g_mmDens = dim2( g_NNmx, g_NNmy );		// time averaged meanDens
	const g_mmTemp = dim2( g_NNmx, g_NNmy );		// time averaged meanTemp
	const g_mmPres = dim2( g_NNmx, g_NNmy );		// time averaged meanPres
	const g_mmVx = dim2( g_NNmx, g_NNmy );			// time averaged meanVx
	const g_mmVy = dim2( g_NNmx, g_NNmy );			// time averaged meanVy

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dim2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Float64Array( nj );
		}
		return a;
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dimInt3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] =  new Int32Array( nk );
			}
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition() {
		g_sysTime = 0.0;
		setCellAttribute();
		setRLGParticles(g_NNp);
		clearMeanField();
	}

	function setCellAttribute() {
		const nsx=g_Nsx, nsy=g_Nsy;
		for (let ic=0; ic<nsx; ic++) {
			for (let jc=0; jc<nsy; jc++) {
				g_cellAttribute[ic][jc] = 0;
				if (ic>=150 && ic<160 && jc>=nsy/2-30 && jc<=nsy/2+30) {
					g_cellAttribute[ic][jc] = 1;
				}
			}
		}
	}

	function setRLGParticles(nn) {
		let i = 0;
		while (i<nn) {
			const x = g_xMax*Math.random();
			const y = g_yMax*Math.random();
			if (cellAttributeAt(x,y)==0) {
				g_xx[i] = x;
				g_yy[i] = y;
				g_vx[i] = 2.0*(Math.random()-0.5)+g_vx0;
				g_vy[i] = 2.0*(Math.random()-0.5);
				i += 1;
			}
		}
	}

	function clearMeanField() {
		const nnmx=g_NNmx, nnmy=g_NNmy;
		for (let i=0; i<nnmx; i++) {
			for (let j=0; j<nnmy; j++) {
				g_meanDens[i][j] = 0.0;
				g_meanVx[i][j] = 0.0;
				g_meanVy[i][j] = 0.0;
				g_meanTemp[i][j] = 0.0;
				g_meanPres[i][j] = 0.0;

				g_mmVx[i][j] = 0.0;
				g_mmVy[i][j] = 0.0;
				g_mmDens[i][j] = 0.0;
				g_mmTemp[i][j] = 0.0;
				g_mmPres[i][j] = 0.0;
			}
		}
	}


	// --------------------  time evolution  --------------------

	function moveParticles(nCalc) {
		for (let i=0; i<nCalc; i++) {
			movement(g_timeStep);
			collision();
			fan(g_vx0);
			g_sysTime += g_timeStep;
		}
		setMean();
		setTimeAverage();
		if (g_sysTime<2*nCalc*g_timeStep) {
			for (let i=0; i<10; i++) setTimeAverage();
		}
	}

	function movement(dt) {
		const nnp=g_NNp;
		for (let i=0; i<nnp; i++) {
			g_vy[i] += g_gravity*dt;
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
			const ca = cellAttributeAt(g_xx[i],g_yy[i]);
			if (ca>0) { // not free space
				g_xx[i] -= g_vx[i]*dt;
				g_yy[i] -= g_vy[i]*dt;
				let r = 1.0;
				if (g_refTemp[ca]>0.0) {
					const temp = 0.5*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
					r = Math.sqrt(g_refTemp[ca]/temp)*(1.5-Math.random());
				} else {
					r = 1.0;
				}
				g_vx[i] = -r*g_vx[i];
				g_vy[i] = -r*g_vy[i];
			}
		}

		for (let i=0; i<nnp; i++) { // x-periodic condition
			if (g_xx[i] < 0.0) {
				g_xx[i] += g_xMax;
			}
			if (g_xx[i] > g_xMax) {
				g_xx[i] -= g_xMax;
			}
			if (g_yy[i] < 0.0) {
				g_yy[i] = 0.0; g_vy[i] = -g_vy[i];
			}
			if (g_yy[i] > g_yMax) {
				g_yy[i] = g_yMax; g_vy[i] = -g_vy[i];
			}
		}
	}

	function cellAttributeAt(x, y) {
		let ix = Math.floor(x/g_dx); if (ix>=g_Nsx) ix = g_Nsx-1;
		if (ix<0) ix = 0;
		let iy = Math.floor(y/g_dy); if (iy>=g_Nsy) iy = g_Nsy-1;
		if (iy<0) iy = 0;
		return g_cellAttribute[ix][iy];
	}

	function collision() {
		const nsx=g_Nsx, nsy=g_Nsy;
		setSection();
		for (let ic=0; ic<nsx; ic++) {
			for (let jc=0; jc<nsy; jc++) {
				const nn = g_section[ic][jc][0];
				if (nn>1) {
					if (g_cellAttribute[ic][jc]==0) {
						collisionInTheCell(ic,jc);
					}
				}
			}
		}
	}

	function collisionInTheCell(ic,jc) {
		const cTh = 0.0;
		const sTh = (Math.random()<0.5) ? -1.0 : 1.0;
		const n = g_section[ic][jc][0];
		let vxm = 0.0, vym = 0.0;
		for (let i=1; i<=n; i++) {
			const k = g_section[ic][jc][i];
			vxm += g_vx[k]; vym += g_vy[k];
		}
		vxm = vxm/n; vym = vym/n;
		for (let i=1; i<=n; i++) {
			const k = g_section[ic][jc][i];
			const vxs = g_vx[k]-vxm;
			const vys = g_vy[k]-vym;
			g_vx[k] = vxm +cTh*vxs +sTh*vys;
			g_vy[k] = vym -sTh*vxs +cTh*vys;
		}
	}

	function setSection() {
		const nsx=g_Nsx, nsy=g_Nsy, nns=g_NNs, nnp=g_NNp;
		for (let i=0; i<nsx; i++) {
			for (let j=0; j<nsy; j++) {
				g_section[i][j][0] = 0;
			}
		}
		for (let ip=0; ip<nnp; ip++) {
			let i = Math.floor(nsx*g_xx[ip]/g_xMax); if (i>=nsx) i = nsx-1;
			let j = Math.floor(nsy*g_yy[ip]/g_yMax); if (j>=nsy) j = nsy-1;
			const iq = g_section[i][j][0]+1;
			if (iq<nns) {
				g_section[i][j][0] = iq;
				g_section[i][j][iq] = ip;
			}
		}
	}

	function maxSection() {
		const nsx=g_Nsx, nsy=g_Nsy;
		let m=0;
		for (let i=0; i<nsx; i++) {
			for (let j=0; j<nsy; j++) {
				if (g_section[i][j][0]>m) m = g_section[i][j][0];
			}
		}
		return m;
	}

	function fan(vx0) {
		const nsy=g_Nsy;
		for (let ic=0; ic<5; ic++) {
			for (let jc=2; jc<nsy-2; jc++) {
				const n = g_section[ic][jc][0];
				for (let ip=1; ip<=n; ip++) {
					const k = g_section[ic][jc][ip];
					g_vx[k] = 2.0*(Math.random()-0.5)+vx0;
					g_vy[k] = 2.0*(Math.random()-0.5);
				}
			}
		}
	}

	// --- statistics

	function totalKineticEnergy() {
		const nnp=g_NNp;
		let tke = 0.0;
		for (let i=0; i<nnp; i++) {
			tke += 0.5*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
		return tke;
	}

	function systemTemperature() {
		return totalKineticEnergy()/g_NNp;
	}

	function viscosityMin(temp) {
		const rho = g_NNp/(1.0*g_Nsx*g_Nsy);
		return (1.0/12.0+0.5*temp*(1-Math.exp(-rho)/(rho-1+Math.exp(-rho))));
	}

	function vorticity(i, j) {
		if (i==0 || i==g_NNmx-1 || j==0 || j==g_NNmy-1) return 0.0;
		return (g_mmVy[i+1][j]-g_mmVy[i-1][j])/(2*g_dx)-(g_mmVx[i][j+1]-g_mmVx[i][j-1])/(2*g_dy);
	}

	function setMean() {
		const nsx=g_Nsx, nsy=g_Nsy, nStep = 5;
		for (let ic=0; ic<nsx; ic+=nStep) {
			for (let jc=0; jc<nsy; jc+=nStep) {
				const ii = Math.floor(ic/nStep);
				const jj = Math.floor(jc/nStep);
				let n = 0, mvx = 0.0, mvy = 0.0;
				for (let i=ic; i<ic+nStep; i++) {
					for (let j=jc; j<jc+nStep; j++) {
						const nn = g_section[i][j][0];
						n = n + nn;
						for (let ipp=1; ipp<nn; ipp++) {
							const k = g_section[i][j][ipp];
							mvx = mvx+g_vx[k];
							mvy = mvy+g_vy[k];
						}
					}
				}
				if (n>0 && g_cellAttribute[ic+Math.floor(nStep/2)][jc+Math.floor(nStep/2)]==0) {
					mvx = mvx/n; mvy = mvy/n;
					const dens = (n/(1.0*nStep*nStep))/g_NinCell;
					g_meanDens[ii][jj] = dens;
					g_meanVx[ii][jj] = mvx;
					g_meanVy[ii][jj] = mvy;
					let ke = 0;
					for (let i=ic; i<ic+nStep; i++) {
						for (let j=jc; j<jc+nStep; j++) {
							const nn = g_section[i][j][0];
							for (let ipp=1; ipp<nn; ipp++) {
								const k = g_section[i][j][ipp];
								ke = ke + 0.5*((g_vx[k]-mvx)*(g_vx[k]-mvx)+(g_vy[k]-mvy)*(g_vy[k]-mvy));
							}
						}
					}
					const temp = ke/n;
					const pres = dens*temp;
					g_meanTemp[ii][jj] = temp;
					g_meanPres[ii][jj] = pres;
				} else {
					g_meanDens[ii][jj] = 0.0;
				}
			}
		}
	}

	function setTimeAverage() {
		const nnmx=g_NNmx, nnmy=g_NNmy, a=0.2;

		for (let i=0; i<nnmx; i++) {
			for (let j=0; j<nnmy; j++) {
				g_mmVx[i][j] = (1.0-a)*g_mmVx[i][j]+a*g_meanVx[i][j];
				g_mmVy[i][j] = (1.0-a)*g_mmVy[i][j]+a*g_meanVy[i][j];
				g_mmDens[i][j] = (1.0-a)*g_mmDens[i][j] + a*g_meanDens[i][j];
				g_mmTemp[i][j] = (1.0-a)*g_mmTemp[i][j] + a*g_meanTemp[i][j];
				g_mmPres[i][j] = (1.0-a)*g_mmPres[i][j] + a*g_meanPres[i][j];
			}
		}
	}


	// --------------------  public  --------------------

	return {
		init:				setInitialCondition,	// setInitialCondition()
		evolve:				moveParticles,			// moveParticles( nCalc )

		getSysParam:		function() { return [ g_NNmx, g_NNmy, g_Nsx, g_Nsy, g_xMax, g_NNp, g_timeStep ]; },
		getNow:				function() { return [ g_sysTime, totalKineticEnergy(), maxSection() ]; },
		getVx0:				function() { return g_vx0; },
		getDensity:			function(i,j) { return g_meanDens[i][j]; },
		getPressure:		function(i,j) { return g_meanPres[i][j]; },
		getTemperature:		function(i,j) { return g_meanTemp[i][j]; },
		getVelocity:		function(i,j) { return [ g_meanVx[i][j], g_meanVy[i][j] ]; },
		getAveDensity:		function(i,j) { return g_mmDens[i][j]; },
		getAvePressure:		function(i,j) { return g_mmPres[i][j]; },
		getAveTemperature:	function(i,j) { return g_mmTemp[i][j]; },
		getAveVelocity:		function(i,j) { return [ g_mmVx[i][j], g_mmVy[i][j] ]; },
		getCellAttribute:	function(i,j) { return g_cellAttribute[i*5+2][j*5+2]; },
		getRLGparticle:		function(i) { return [ g_xx[i], g_yy[i], g_vx[i], g_vy[i] ]; },
	};

})(); // ====================  windTunnelRLG2D end  ====================


const js048 = (function(){ // ====================  js Module  ====================

	const theModule = windTunnelRLG2D;
	const xCanvasSize = 560;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_nCalc = 1;

	let p_NNmx, p_NNmy, p_Nsx, p_Nsy, p_xMax, p_NNp, p_timeStep; // <-- theModule.getSysParam();
	let sysTime, totalKineticEnergy, maxSection;
  let pressArray = [];
	let tempArray = [];
  let vxArray = [];
	let vyArray = [];
	let cellKindArray = [];


	let dispMode =  10;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;

  let breakFlag = false;
  let getFieldFlag = true;
	let fieldKind = 1;
	let timeStamp = 0.0;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();
		viewHome();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init(  );
			[ p_NNmx, p_NNmy, p_Nsx, p_Nsy, p_xMax, p_NNp, p_timeStep ] = theModule.getSysParam();
			// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
			g3d.init( p_NNmx, p_NNmy, p_NNmx/2, p_xMax/p_NNmx, xCanvasSize, yCanvasSize, 500, 20 );
			g3d.drawWallGrid2D.threshold = -1;
			getFieldFlag = true;
			fieldKind = 1;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_nCalc );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_nCalc );
		}

		draw( ctx, dispMode );

		if ( getFieldFlag ) setFieldData( fieldKind );

		requestAnimationFrame(animate);
	}

  function setFieldData( fieldKind ) {
    if (fieldKind==1) {
      for (let i=0; i<p_NNmx; i++) {
        fieldArray[i] = [];
        vectorArray[i] = [];
        for (let j=0; j<p_NNmy; j++) {
          fieldArray[i][j] = theModule.getAvePressure(i,j);
          vectorArray[i][j] = theModule.getAveVelocity(i,j);
        }
      }
    }
  }

  function setFieldData( fieldKind ) {
    if (fieldKind==1) {
			pressArray = [];
			tempArray = [];
			vxArray = [];
			vyArray = [];
			cellKindArray = [];
			timeStamp = sysTime;
      for (let i=0; i<p_NNmx; i++) {
				pressArray[i] = [];
				tempArray[i] = [];
				vxArray[i] = [];
				vyArray[i] = [];
				cellKindArray[i] = [];
        for (let j=0; j<p_NNmy; j++) {
          pressArray[i][j] = theModule.getAvePressure(i,j);
					tempArray[i][j] = theModule.getAveTemperature(i,j);
					let vx, vy;
          [ vx, vy ] = theModule.getAveVelocity(i,j);
					vxArray[i][j] = vx;
					vyArray[i][j] = vy;
					cellKindArray[i][j] = theModule.getCellAttribute(i,j); // 0:free, 1:fixed
        }
      }
    }
  }


	//--------------------  draw  --------------------

	function draw( ctx, dispMode ) {
		const wallFunc = function(i,j) { return (theModule.getCellAttribute(i,j)==1); };
		const densFunc = function(i,j) { return 50.0*theModule.getAveDensity(i,j)-50; };

		const NNmx = p_NNmx, NNmy = p_NNmy, xMax = p_xMax, NNp = p_NNp, timeStep = p_timeStep;
		const xBoxSize = 500;
		const xp = 30, yp=340, sc = xBoxSize/NNmx, yTextPos= 435;
		const vx0 = theModule.getVx0();

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		if ( dispMode<=8 ) drawBox( ctx, NNmx, NNmy, xp, yp, sc );

		if (dispMode<=2) { // 0:density(x,y), 1:temperature(x,y), 2:pressure(x,y)
			drawField(ctx, NNmx, NNmy, xp, yp, sc, dispMode );
			drawFlow(ctx, NNmx, NNmy, xp, yp, sc);
		} else if (dispMode==3) { // 3: flow(x,y)
			drawFlow(ctx, NNmx, NNmy, xp, yp, sc);
		} else if ( dispMode==4 || dispMode==5 || dispMode==6) { // time averaged 4:density, 5:temperature, 6:pressure
			drawTimeAveragedField( ctx, NNmx, NNmy, xp, yp, sc, dispMode );
			drawTimeAveragedFlow( ctx, NNmx, NNmy, xp,yp,sc );
		} else if (dispMode==7) { // 7: time averaged flow(x,y)
			drawTimeAveragedFlow( ctx, NNmx, NNmy, xp,yp,sc );
		}  else if (dispMode==8) { // 8:RLG particles - sample 500
			drawSample( ctx, xBoxSize/xMax, timeStep, 500, xp, yp );

		} else if (dispMode==9) {
			// g3d.drawWallGrid2D( ctx, rotAngle, wallFunc, zFunc, colorFactor, inc [, showBox] )
			g3d.drawWallGrid2D( ctx, 0.0, wallFunc, densFunc, 0.5, 1 );
		} else if (dispMode==10) {
			const vFunc = function(i,j) {
				const mag = 50;
				let mvx,mvy;
				[ mvx, mvy ] = theModule.getAveVelocity(i,j);
				return [ mvx*mag, mvy*mag ];
			}
			// g3d.drawWallVectorField2D(ctx, rotAngle, wallFunc, zFunc, vFunc, colorMode, inc [, showBox] )
			g3d.drawWallVectorField2D( ctx, 0.0, wallFunc, densFunc, vFunc, 0, 1 );
		}

		[ sysTime, totalKineticEnergy, maxSection ] = theModule.getNow();
		ctx.fillStyle = "#888888";
		ctx.fillText(`periodic box = ${NNmx} x ${NNmy}`, xp, yTextPos);
		ctx.fillText(`time = ${sysTime.toFixed(1)}`, xp+200, yTextPos);
		ctx.fillText(`RLG particle N = ${NNp}`, xp, yTextPos+20);
		ctx.fillText(`kinetic energy = ${totalKineticEnergy.toFixed(4)}`, xp+200, yTextPos+20);
		ctx.fillText(`max section = ${maxSection}`, xp, yTextPos+40);
	}

	function drawBox(ctx, nnmx, nnmy, xp,yp,sc) {
		ctx.fillStyle = "#999999";
		ctx.fillRect(xp-1, yp-nnmy*sc-1, nnmx*sc+2, nnmy*sc+2);
		// draw wall
		for (let i=0; i<nnmx; i++) {
			for (let j=0; j<nnmy; j++) {
				if ( theModule.getCellAttribute(i,nnmy-j-1)==1 ) {
					drawRect(ctx, xp+i*sc+0,yp-j*sc-5-4,8,8,"#404040");
				}
			}
		}
	}

	function drawSample(ctx, scale, timeStep, n,xp,yp) {

		for (let i=0; i<n; i++) {
			let xi, yi, vxi, vyi;
			[ xi, yi, vxi, vyi ] = theModule.getRLGparticle(i);
			ctx.strokeStyle = "rgb(0,0,250)";
			ctx.beginPath();
			ctx.arc( xp+xi*scale, yp-yi*scale, 5, 0, 2*Math.PI, false );
			ctx.stroke();

			ctx.strokeStyle = "rgb(250,0,0)";
			ctx.beginPath();
			ctx.moveTo( xp+xi*scale, yp-yi*scale );
			ctx.lineTo( xp+(xi+vxi*10*timeStep)*scale, yp-(yi+vyi*10*timeStep)*scale );
			ctx.stroke();
		}
	}

	function drawFlow(ctx, nnmx, nnmy, xp,yp,sc) {
		let mag = 80;

		for (let i=0; i<nnmx; i++) {
			for (let j=0; j<nnmy; j++) {
				const densij = theModule.getDensity(i,j);
				if (densij>0) {
					let vxij, vyij;
					[ vxij, vyij ] = theModule.getVelocity(i,j);
					const color = ( vxij>=0 ) ? "rgb(0,0,250)" : "rgb(220,0,120)";
					//color = (g_meanVy[i][j]>=0) ? "rgb(250,0,0)" : "rgb(120,0,250)";
					let x = (i+0.5)*sc
					let y = (j+0.5)*sc;
					g3d.drawLine(ctx, xp+x,yp-y,xp+x+vxij*mag,yp-y-vyij*mag,color);
				}
			}
		}
	}

	function drawTimeAveragedFlow(ctx, nnmx, nnmy, xp, yp, sc) {
		let mag = 80;

		for (let i=0; i<nnmx; i++) {
			for (let j=0; j<nnmy; j++) {
				const densij = theModule.getAveDensity(i,j);
				if ( densij>0 ) {
					let mmvxij, mmvyij;
					[ mmvxij, mmvyij ] = theModule.getAveVelocity(i,j);
					let color = ( mmvxij>=0 ) ? "rgb(0,0,250)" : "rgb(220,0,120)";
					//color = (g_meanVy[i][j]>=0) ? "rgb(250,0,0)" : "rgb(120,0,250)";
					let x = (i+0.5)*sc;
					let y = (j+0.5)*sc;
					g3d.drawLine(ctx, xp+x,yp-y,xp+x+mmvxij*mag,yp-y-mmvyij*mag,color);
				}
			}
		}
	}

	function drawField( ctx, nnmx, nnmy, xp, yp, sc, dispMode ) {
		const dmag =4.0
		let t, p, color;

		for (let i=0; i<nnmx; i++) {
			for (let j=0; j<nnmy; j++) {
				const x = (i+0.5)*sc, y = (j+0.5)*sc;
				const r = Math.min( theModule.getDensity(i,j)*dmag, 0.8*sc );
				if (r>=1) {
					if (dispMode==0) {
						color = "rgb(0,250,0)";
					} else if (dispMode==1) {
						t = Math.max(120-(theModule.getTemperature(i,j)-0.333)*500,0);
						t = Math.floor(Math.min(t,240));
						color = "hsl("+t+",100%,50%)";
					} else if (dispMode==2) {
						p = Math.max(120-(theModule.getPressure(i,j)-0.333)*200,0);
						p = Math.floor(Math.min(p,240));
						color = "hsl("+p+",100%,50%)";
					}
					g3d.drawDisc(ctx, xp+x,yp-y,r,color);
				}
			}
		}
	}

	function drawTimeAveragedField( ctx, nnmx, nnmy, xp, yp, sc, dispMode ) {
		const dmag = 4.0
		let t, p, color;

		for (let i=0; i<nnmx; i++) {
			for (let j=0; j<nnmy; j++) {
				const x = (i+0.5)*sc, y = (j+0.5)*sc;
				const r = Math.min( theModule.getAveDensity(i,j)*dmag, 0.8*sc );
				if (r>=1) {
					if (dispMode==4) {
						color = "rgb(0,250,0)";
					} else if (dispMode==5) {
						t = Math.max(120-(theModule.getAveTemperature(i,j)-0.333)*500,0);
						t = Math.floor(Math.min(t,240));
						color = "hsl("+t+",100%,50%)";
					} else if (dispMode==6) {
						p = Math.max(120-(theModule.getAvePressure(i,j)-0.333)*200,0);
						p = Math.floor(Math.min(p,240));
						color = "hsl("+p+",100%,50%)";
					}
					g3d.drawDisc(ctx, xp+x,yp-y,r,color);
				}
			}
		}
	}

	function drawRect(ctx, x, y, w, h, color) {
		ctx.fillStyle = color;
		ctx.fillRect(x, y, w, h);
	}


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------

	// g3d_extension grid2d  created 2023.06.17, last updated 2023.06.17
	// g3d.drawWallGrid2D( ctx, rotAngle, wallFunc, zFunc, colorFactor, inc [, showBox] )
	g3d.drawWallGrid2D = function ( ctx, rotAngle, wallFunc, zFunc, colorFactor, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx, nny = g_NNy, threshold = g3d.drawWallGrid2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let jj=0; jj<nny; jj+=inc) {
			let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
			for (let ii=0; ii<nnx; ii++) {
				let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
				if (i<0 || i+1>nnx-1) continue;

				if ( wallFunc(i,nny-j-1) ) {
					const  x = i*g_dx, y = j*g_dx, z = 0, z1 = 2*g3d.cz0;
					g3d.drawRotatedLine(ctx, x,y,z,x,y,z1,'#888888',sc,xp,yp);
					continue;
				}

				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i+1,nny-j-1);
				const x1 = (i+1)*g_dx, y1 = j*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				if (j<0 || j+1>nny-1) continue;

				if ( wallFunc(i,nny-j-1) ) {
					continue;
				}

				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i,nny-j-2);
				const x1 = i*g_dx, y1 = (j+1)*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawWallGrid2D.threshold = 0.005;

	// g3d_extension gvextor field  created 2023.06.17, last updated 2023.06.17
	// g3d.drawWallVectorField2D(ctx, rotAngle, wallFunc, zFunc, vFunc, colorMode, inc [, showBox] )
	g3d.drawWallVectorField2D = function(ctx, rotAngle, wallFunc, zFunc, vFunc, colorMode, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, threshold = g3d.drawWallVectorField2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii; if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj+=inc) {
				let j=jj; if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;

				if ( wallFunc(i,nny-j-1) ) {
					const  x = i*dx, y = j*dx, z = 0, z1 = 2*g3d.cz0;
					g3d.drawRotatedLine(ctx, x,y,z,x,y,z1,'#888888',sc,xp,yp);
					continue;
				}

				const f = zFunc(i,nny-j-1);
				const x = i*dx, y = j*dx, z = f + g3d.cz0;
				let vx, vy;
				[ vx, vy ] = vFunc(i,nny-j-1);
				if ( vx*vx+vy*vy < threshold ) continue;
				const x1 = x + vx, y1 = y -vy, z1 = z;
				let colr;
				if ( colorMode==0 ) {
					const hue = Math.floor((Math.atan2(vy,vx))*180.0/Math.PI+180.0);
					colr = `hsl(${hue},100%,50%)`;
				} else if ( colorMode==1 ) {
					colr = (vx>0) ? "#3333ff" : "#ff3333";
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawWallVectorField2D.threshold = 0.5;


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	/*
	function setTheme() {
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}
	*/

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle(15,-10);
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, totalKineticEnergy, maxSection ];
  }

  function pygetFieldData() {
		fieldKind = 0;
    return [ timeStamp, pressArray, tempArray, vxArray, vyArray, cellKindArray ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		//setTheme:		setTheme,		// setTheme()
		setDispMode:	setDispMode,	// setDispMode()
		viewHome:		viewHome,		// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, totalKineticEnergy, maxSection ]
		pygetFieldData: pygetFieldData, // pygetFieldData() : return [ timeStamp, pressArray, tempArray, vxArray, vyArray, cellKindArray ]
	};

})(); // ====================  jsc048 module end  ====================


const js = js048;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js048] wind tunnel - periodic real coded lattice gas model 2D</p>
<canvas id="canvas_box" style="background-color: #000000;" width="560" height="480"></canvas>
<br>

theme: wind tunnel
    <span style="margin-right: 180px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>r:density,color:green</option><option>color:temperature,r:density</option>
<option>color:pressure, r:density</option><option>flow</option>
<option>r:time-averaged density</option><option>color:time-averaged temperature, r:density</option>
<option>color:time-averaged pressure, r;density</option><option>time-averaged flow</option>
<option>RLG particles - sample500</option>
<option>grid2d: time-averaged density</option>
<option selected>grid2d: time-averaged flow z:density</option>
</select>
<br>

<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="560" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def



In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ sysTime, totalKineticEnergy, maxSection ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f'i = {i:>2d},  time = {sysTime:>8.1f},  total kinetic energy = {totalKineticEnergy:>10.2f}')
  time.sleep(2)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# change dispMode

import time

dispModeList = [
    '0: r-density,color-green', '1: r-density,color-temperature', '2: r-density,color-pressure', '3: flow',
    '4: r-time_averaged density', '5: color-time_averaged temperature', '6: color-time_averaged pressure', '7: color-time_averaged flow',
    '8: RLG particles - sample500', '9: grid2d: time-averaged density', '10: grid2d: time-averaged flow z:density' ]

# exec html-js code
exec_html_js()
print("-- start --")

# change dispMode and print
for dispMode in [ 4, 5, 6, 7, 8, 9, 10 ]:
  eval_js( 'js.pysetDispMode({})'.format(dispMode) )
  print("-- dispMode:", dispModeList[dispMode], " --")
  [ sysTime, totalKineticEnergy, maxSection ] = eval_js( 'js.pygetData({})'.format(dispMode) )
  print( "\t time = {:>8.1f},  total kinetic energy = {:>10.2f} ".format( sysTime, totalKineticEnergy ) )
  time.sleep(3)

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# get field data

import time
import numpy as np

# exec html-js code
exec_html_js()
print("-- start --")
print("-- just a moment ( 30 sec.) --")
time.sleep(30)

# get field and vector data
print("-- get field data --")
[ timeStamp, pressArray, tempArray, vxArray, vyArray, cellKindArray ] = eval_js('js.pygetFieldData()')
print("-- OK --")

# animation break to END
eval_js('js.breakLoop()')
print("-- simulator stop --")

# convert np array
Press = np.array(pressArray)
Temp = np.array(tempArray)
Vx = np.array(vxArray)
Vy = np.array(vyArray)
Knd = np.array(cellKindArray)
print("time stamp =",round(timeStamp,2) )
print(f"field array shape - Press:{Press.shape},  Temp:{Temp.shape},  Vx:{Vx.shape}, Vy:{Vy.shape}, Knd:{Knd.shape}")
print("-- end --")



In [ ]:
# save field data as numpy file

import numpy as np

print("-- save np.array([ pressArray, tempArray, vxArray, vyArray, cellKindArray ]) as 'js048_data.npy' --")
np_data = np.array([ pressArray, tempArray, vxArray, vyArray, cellKindArray ])
print("np_data.shape", np_data.shape)

np.save('js048_data.npy :', np_data)

In [ ]:
# plot pressure(x,y) - map

import numpy as np
import matplotlib.pyplot as plt

Z = Press.T
im = plt.imshow(Z, origin='lower', cmap='jet' )
plt.colorbar(im)
plt.title("pressure map")
plt.show()

In [ ]:
# plot pressure(x,y)

import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')
nx, ny = Press.shape
x = np.arange(0,nx)
y = np.arange(0,ny)
X, Y = np.meshgrid(x, y)
Z = Press.T
ax.plot_surface(X,Y,Z, cmap='jet')
ax.set_box_aspect((1,ny/nx,0.5))
plt.xlabel("x")
plt.ylabel("y")
plt.title("pressure")
plt.show()

In [ ]:
# def surface_plotly( FieldArray, titleString ) and plot

import numpy as np
import plotly.graph_objects as go

def surface_plotly( FieldArray, titleString ):
  nx, ny = FieldArray.shape

  # prepare for surface plot in plotly
  fig = go.Figure(data=[go.Surface(z=FieldArray.T, x=np.arange(0,nx), y=np.arange(0,ny))])
  fig.update_layout(
      title=titleString,
      width=800, height=800,
      scene_aspectratio=dict(x=1, y=ny/nx, z=0.5),
      scene_camera_eye=dict(x=0, y=-1.5, z=0.5))
  fig.show()

surface_plotly( Press, 'Pressure(x,y)' )

In [ ]:
# flow V(x,y) plot

import numpy as np
import matplotlib.pyplot as plt

nx, ny = Press.shape
x = np.arange(0,nx)
y = np.arange(0,ny)
X, Y = np.meshgrid(x, y)
U = (Vx.T).flatten()
V = (Vy.T).flatten()

# vector field plot
fig = plt.figure(figsize=(12,7))
ax1 = fig.add_subplot(111)

scale_factor = 4
ax1.quiver(X, Y, U, V, scale=scale_factor )
plt.title("flow vector (vx,vy)")
plt.show()